In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from urllib.parse import urlparse

class MITREExtractor:
    #necessary components, the init method, just captures the url and setting arguments for the upcoming methods
    def __init__(self, url, max_pages=20):
        self.driver = webdriver.Chrome() 
        self.driver.get(url)
        self.max_pages = max_pages
        self.page_count = 0
        self.base_domain = urlparse(url).netloc

    def extract_text_data(self):
        text_data = ""
        body_element = self.driver.find_element(By.TAG_NAME, 'body')
        text_data = body_element.text
        return text_data

    def find_next_visible_link(self):
        links = self.driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            if link.is_displayed():
                href = link.get_attribute('href')
                if href and urlparse(href).netloc == self.base_domain:
                    return link
        return None

    def extract_and_save(self, output_file):
        while self.page_count < self.max_pages:
            # Extract text data from the current page
            page_text_data = self.extract_text_data()
            
            # Append the text data to a file
            with open(output_file, 'a', encoding='utf-8') as file:
                file.write("%s\n\n" % page_text_data)
            
            # Find the next visible link within the same domain and click it
            next_link = self.find_next_visible_link()
            if next_link:
                next_link.click()
                self.page_count += 1
                time.sleep(2)  # Wait for the next page to load
            else:
                print("No more visible links within the domain to follow.")
                break

        # Close the WebDriver
        self.driver.quit()

# Usage
extractor = MITREExtractor("https://attack.mitre.org/")
extractor.extract_and_save('extracted_text_data.txt')

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (optional)

# Set up the Chrome WebDriver (update the path to where you saved chromedriver)
service = Service('/path/to/chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the MITRE ATT&CK website
url = "https://attack.mitre.org/"
driver.get(url)

# Wait for the page to load (adjust the time if needed)
time.sleep(5)

# Find all 'a' tags
links = driver.find_elements(By.TAG_NAME, 'a')

# Extract href and text from each link
extracted_links = []
for link in links:
    href = link.get_attribute('href')
    text = link.text
    if href and text:
        extracted_links.append(f"{text}: {href}")

# Close the browser
driver.quit()

# Save the extracted links to a text file
with open('mitre_attack_links.txt', 'w', encoding='utf-8') as f:
    for link in extracted_links:
        f.write(link + '\n')

print(f"Extracted {len(extracted_links)} links and saved to mitre_attack_links.txt")

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
